# LSSTCam pointing errors
## This version just does the on-sky pointing error.

Craig Lage - 17-Nov-25

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord, angular_separation
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors
import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTCam")
instrument = 'LSSTCam'
camera = LsstCam.getCamera()
client = makeEfdClient()

## Get the data

In [ ]:
startDay = 20251119
endDay = 20251119

detector = camera['R22_S11']
bbox = detector.getBBox()

els = []
azs = []
point_ras = []
point_decs = []
true_ras = []
true_decs = []
dayObs = startDay
expIds = []
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            calExp = butler.get('preliminary_visit_image', detector=94, visit=record.id, instrument=instrument)
            cWcs = calExp.getWcs()
            if cWcs == None:
                print(f"{record.id} had no cWcs.")
                continue
            rawExp = butler.get('raw', detector=94, exposure=record.id, instrument=instrument)
            md = rawExp.getMetadata()
            calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
            true_ra = calExpSkyCenter.getRa().asDegrees()
            true_dec = calExpSkyCenter.getDec().asDegrees()
            expIds.append(record.id)
            true_ras.append(true_ra)
            true_decs.append(true_dec)
            point_ra = md['RASTART']
            point_dec = md['DECSTART']
            point_ras.append(point_ra)
            point_decs.append(point_dec)
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
    print(dayObs, len(true_ras))
    dayObs = calcNextDay(dayObs)


In [ ]:
separation = []
for i in range(len(point_ras)):
    sep = angular_separation(point_ras[i] * u.deg, point_decs[i] * u.deg, 
                             true_ras[i] * u.deg, true_decs[i] * u.deg)
    # Returns in radians
    separation.append(sep.value * 180.0 / np.pi * 3600.0)
lt_40 = [sep for sep in separation if sep < 40]
pct_lt_40 = len(lt_40) / len(separation) * 100    
fig, ax = plt.subplots(1, 1, figsize=(10,5))
plt.suptitle(f"On sky pointing model error, {startDay}-{endDay}", fontsize=18)
ax.hist(separation, bins = 50, range=(0,200), alpha=0.5)
ax.text(125, 200, f"Median = {np.nanmedian(separation):.2f}")
ax.text(125, 150, f"Median = {np.nanmedian(pct_lt_40):.1f} % of visits\n are < 40 arcseconds")
ax.set_xlim(0, 200)
ax.set_xlabel("Error (arcseconds)")
    
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/OnSky_Pointing_Errors_{startDay}-{endDay}.png")

In [ ]:
dayObs = startDay
separation = []
seqNums = []
rot_angles = []
pos = []
for i in range(len(point_ras)):
    sep = angular_separation(point_ras[i] * u.deg, point_decs[i] * u.deg, 
                             true_ras[i] * u.deg, true_decs[i] * u.deg)
    # Returns in radians
    separation.append(sep.value * 180.0 / np.pi * 3600.0)
    seqNums.append(int(expIds[i] - dayObs*1E5))
    rot_angles.append(rots[i])
    if azs[i] < 150:
        pos.append(2)
    else:
        pos.append(1)
    
fig, axes = plt.subplots(1, 3, figsize=(16,5))
plt.suptitle(f"On sky pointing model error, {dayObs}", fontsize=18)
axes[0].hist(separation, bins = 50, range=(0,200), alpha=0.5)
axes[0].text(120, 25, f"Median = {np.nanmedian(separation):.2f}")
axes[0].set_xlim(0, 200)
axes[0].set_xlabel("Error (arcseconds)")
axes[1].scatter(seqNums, separation)
axes[1].set_xlabel("Sequence number")
axes[1].set_ylabel("Error (arcseconds)")
axes[2].scatter(rot_angles, separation)
axes[2].set_xlabel("Rotator physical angle (degrees)")
axes[2].set_ylabel("Err(or (arcseconds)")
    
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/OnSky_Pointing_Errors_{dayObs}.png")